In [ ]:
!pip install -U transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 128.7 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.56.0
    Uninstalling transformers-4.56.0:
      Successfully uninstalled transformers-4.56.0


In [ ]:
# Colab cell
!pip -q install -U transformers datasets peft accelerate evaluate
# optional (only if you plan to try QLoRA / 4-bit quantization later)
!pip -q install -U bitsandbytes


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.4 MB/s eta 0:00:00
^C
^C


In [ ]:
# Colab cell (Python)
import torch
print("torch:", torch.__version__, "cuda:", torch.cuda.is_available())


torch: 2.8.0+cu126 cuda: True


In [ ]:
from datasets import load_dataset
ds = load_dataset("AkashPS11/recipes_data_food.com", split="train")
print(ds.num_rows)
print(ds.column_names)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

recipes.csv:   0%|          | 0.00/34.1M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1048543 [00:00<?, ? examples/s]

1048543
['RecipeId', 'Barcode', 'Name', 'AuthorId', 'AuthorName', 'CookTime', 'PrepTime', 'TotalTime', 'DatePublished', 'Description', 'Images', 'RecipeCategory', 'Keywords', 'RecipeIngredientQuantities', 'RecipeIngredientParts', 'AggregatedRating', 'ReviewCount', 'Calories', 'FatContent', 'SaturatedFatContent', 'CholesterolContent', 'SodiumContent', 'CarbohydrateContent', 'FiberContent', 'SugarContent', 'ProteinContent', 'RecipeServings', 'RecipeYield', 'RecipeInstructions']


In [ ]:
def join_list(x):
    if x is None: return ""
    if isinstance(x, list):
        return ", ".join([str(t).strip() for t in x if t])
    return str(x).strip()

def make_text(example):
    name = example.get("Name","").strip()
    ing  = join_list(example.get("RecipeIngredientParts"))
    instr = example.get("RecipeInstructions") or ""
    if isinstance(instr, list):
        instr = " ".join([s.strip() for s in instr if s])
    instr = str(instr).strip()
    prompt = f"Recipe: {name}\nIngredients: {ing}\nInstructions:"
    # we concatenate prompt + " " + target to make a single sequence
    return {"text": prompt + " " + instr}

ds = ds.filter(lambda r: r.get("Name") and r.get("RecipeIngredientParts") and r.get("RecipeInstructions"))
ds = ds.map(make_text, remove_columns=ds.column_names)


Filter:   0%|          | 0/1048543 [00:00<?, ? examples/s]

Map:   0%|          | 0/1228 [00:00<?, ? examples/s]

In [ ]:
ds = ds.train_test_split(test_size=0.01, seed=42)
train_ds = ds["train"].shuffle(seed=42).select(range(min(5000, len(ds["train"]))))
val_ds   = ds["test"]


In [ ]:
from transformers import AutoTokenizer
MODEL_NAME = "distilgpt2"   # beginner friendly & small

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

def tokenize_fn(batch):
    return tokenizer(batch["text"], truncation=True, max_length=512)

train_tok = train_ds.map(tokenize_fn, batched=True, remove_columns=["text"])
val_tok   = val_ds.map(tokenize_fn, batched=True, remove_columns=["text"])


Map:   0%|          | 0/1215 [00:00<?, ? examples/s]

Map:   0%|          | 0/13 [00:00<?, ? examples/s]

In [ ]:
import torch
from transformers import AutoModelForCausalLM
from peft import LoraConfig, get_peft_model

base_model = AutoModelForCausalLM.from_pretrained(MODEL_NAME)

lora_config = LoraConfig(
    r=8,                # rank (small low-rank dimension)
    lora_alpha=16,      # scaling
    lora_dropout=0.1,
    target_modules=["c_attn","c_proj","c_fc"],   # common GPT2 targets
    bias="none",
    task_type="CAUSAL_LM",
)

model = get_peft_model(base_model, lora_config)
model.print_trainable_parameters()  # confirm only LoRA params are trainable


trainable params: 589,824 || all params: 82,502,400 || trainable%: 0.7149


/usr/local/lib/python3.12/dist-packages/peft/tuners/lora/layer.py:2174: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


In [ ]:
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling
import math

data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

training_args = TrainingArguments(
    output_dir="lora_recipes_out",
    num_train_epochs=2,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=8,    # effective batch size = 2*8 = 16
    learning_rate=2e-4,
    fp16=True,                        # use mixed precision if GPU supports it
    eval_strategy="steps",
    eval_steps=200,
    logging_steps=50,
    save_steps=200,
    save_total_limit=2,
    remove_unused_columns=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tok,
    eval_dataset=val_tok,
    data_collator=data_collator,
)

trainer.train()
eval_metrics = trainer.evaluate()
print("Eval loss:", eval_metrics["eval_loss"], "Perplexity:", math.exp(eval_metrics["eval_loss"]))

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: a-redlahansika (a-redlahansika-keshav-memorial-college-of-law) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


`loss_type=None` was set in the config but it is unrecognized. Using the default loss: `ForCausalLMLoss`.


Eval loss: 2.7750818729400635 Perplexity: 16.03994017895874


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from peft import PeftModel

model_name = "distilgpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Fix: set pad_token to eos_token
tokenizer.pad_token = tokenizer.eos_token

# Load base model
model = AutoModelForCausalLM.from_pretrained(model_name)

# Load LoRA adapter
model = PeftModel.from_pretrained(model, "lora_recipes_out/checkpoint-152")

# Create pipeline with fixed settings
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device=0,  # -1 = CPU, change to 0 if using GPU
)

prompt = "Recipe:"
output = pipe(
    prompt,
    max_new_tokens=80,
    do_sample=True,
    top_p=0.9,
    temperature=0.7,
    repetition_penalty=1.2
)

print(output[0]['generated_text'])


Device set to use cuda:0


Recipe: Tender Cheesecake
Ingredients: c("salt", "butter", "-fermented sugar", "...water", "+milk") Directions: c('Combine all ingredients in a bowl and combine until smooth, about 3 minutes.",)  teaspo (or use more salt than you would normally add to the mixture. In an oven or 350F heat for 5-6 minutes.)


To use an API key securely, add it to Colab's secrets manager under the "🔑" in the left panel. Give it the name `WANDB_API_KEY`. Then pass the key to your code as follows: